In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
# sys.path = ['/home/jxm3/research/retrieval/inversion'] + sys.path
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

%load_ext autoreload
%autoreload 2

In [56]:
import analyze_utils

checkpoint_folder = "/home/jxm3/research/retrieval/inversion/saves/98b1418d38c3f9333b17ab20bff06ff9/"
trainer = analyze_utils.load_trainer(checkpoint_folder).inversion_trainer

Loading trainer for analysis – setting --do_eval=1
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set train_args.dataloader_num_workers = 4


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wre
================ End trainer sanity check ================


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [57]:
import torch

In [58]:
eval_batch = next(iter(trainer.get_eval_dataloader(eval_dataset=trainer.eval_dataset["nq"])))
eval_batch = {k: v[None, 0] for k,v in eval_batch.items() } # take first item
eval_batch = {k: v.to(trainer.args.device) for k,v in eval_batch.items() } # put on GPU

In [59]:
trainer.embedder_tokenizer.batch_decode(eval_batch['input_ids'])

['to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact</s>']

In [60]:
(probs @ trainer.model.embedder.get_input_embeddings().weight).shape

torch.Size([1, 32, 768])

In [ ]:
seq_length = 32 - 1
vocab_size = trainer.model.embedder.get_input_embeddings().weight.shape[0]
logits = torch.rand((1, seq_length, vocab_size), device=trainer.args.device, requires_grad=True)
attention_mask = torch.ones((1, seq_length), dtype=torch.long, device=trainer.args.device)
trainer.model.embedder.get_input_embeddings().weight.requires_grad = False


with torch.no_grad():
    true_embeddings = trainer.model.call_embedding_model(
        input_ids=eval_batch["embedder_input_ids"],
        attention_mask=eval_batch["embedder_attention_mask"],
    )

for _name, param in trainer.model.embedder.named_parameters(): param.requires_grad = False

opt = torch.optim.Adam(params=[logits], lr=1e-1)

# tt = trainer.model.embedder_tokenizer('This is a random sentence!', return_tensors='pt')
# tt = tt.to(trainer.args.device)
print('goal text:', trainer.model.embedder_tokenizer.decode(eval_batch['embedder_input_ids'][0]))

eos_token_embed = trainer.model.embedder.embed_tokens(
    torch.tensor([[1]], dtype=torch.long, device=trainer.args.device)
)
eos_token_embed.requires_grad = False

num_steps = 10000
log_interval = 100
tau = 0.5
for step in range(num_steps):
    probs = torch.nn.functional.gumbel_softmax(logits, tau=tau)
    # probs = torch.nn.functional.softmax(logits, -1)
    inputs_embeds = probs @ trainer.model.embedder.get_input_embeddings().weight
    
    inputs_embeds = torch.cat(
        (inputs_embeds, eos_token_embed),
        dim=1
    )
    
    output = trainer.model.embedder(
        inputs_embeds=inputs_embeds
    )
    attention_mask = torch.ones(inputs_embeds.shape[0:2], device=trainer.args.device)
    embeddings = trainer.model._process_embedder_output(output, attention_mask)
    
    # loss = ((embeddings - true_embeddings) ** 2).sum(1)
    loss = 1.0 - torch.nn.CosineSimilarity(1)(true_embeddings, embeddings)
    
    if step % log_interval == 0:
        gen_text = trainer.model.embedder_tokenizer.decode(logits[0].argmax(-1))
        print(f'step {step} loss {loss}')
        print(f'\tgen_text = {gen_text}')
    loss.backward()
    opt.step()
    
    if loss < 1e-3: break

In [102]:
[k for k in dir(trainer.model.embedder) if 'emb' in k]

['_get_resized_embeddings',
 '_named_members',
 '_resize_token_embeddings',
 'embed_tokens',
 'get_input_embeddings',
 'get_output_embeddings',
 'get_position_embeddings',
 'resize_position_embeddings',
 'resize_token_embeddings',
 'set_input_embeddings']

## try a shorter input

In [ ]:
tt = trainer.model.embedder_tokenizer('hi there!', return_tensors='pt')
tt = tt.to(trainer.args.device)

with torch.no_grad():
    true_embeddings = trainer.model.call_embedding_model(
        input_ids=tt["input_ids"],
        attention_mask=tt["attention_mask"],
    )

###
seq_length = tt['input_ids'].shape[1]
vocab_size = trainer.model.embedder.get_input_embeddings().weight.shape[0]
logits = torch.rand((1, seq_length, vocab_size), device=trainer.args.device, requires_grad=True)
attention_mask = torch.ones((1, seq_length), dtype=torch.long, device=trainer.args.device)
trainer.model.embedder.get_input_embeddings().weight.requires_grad = False
opt = torch.optim.Adam(params=[logits], lr=.05)
####

for _name, param in trainer.model.embedder.named_parameters(): param.requires_grad = False
print('goal text:', trainer.model.embedder_tokenizer.decode(tt['input_ids'][0]))


eos_token_embed = trainer.model.embedder.embed_tokens(
    torch.tensor([[1]], dtype=torch.long, device=trainer.args.device)
)
eos_token_embed.requires_grad = False

num_steps = 10000
log_interval = 100
tau = 1
for step in range(num_steps):
    probs = torch.nn.functional.gumbel_softmax(logits, tau=tau)
    # probs = torch.nn.functional.softmax(logits, -1)
    inputs_embeds = probs @ trainer.model.embedder.get_input_embeddings().weight
    inputs_embeds = torch.cat(
        (inputs_embeds, eos_token_embed),
        dim=1
    )
    output = trainer.model.embedder(
        inputs_embeds=inputs_embeds
    )
    attention_mask = torch.ones(inputs_embeds.shape[0:2], device=trainer.args.device)
    embeddings = trainer.model._process_embedder_output(output, attention_mask)
    
    loss = 1.0 - torch.nn.CosineSimilarity(1)(true_embeddings, embeddings)
    
    if step % log_interval == 0:
        gen_text = trainer.model.embedder_tokenizer.decode(logits[0].argmax(-1))
        print(f'step {step} loss {loss}')
        print(f'\tgen_text = {gen_text}')
    loss.backward()
    opt.step()
    
    if loss < 1e-3: break

goal text: hi there!</s>
step 0 loss tensor([0.8693], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = Booksătă Sanchez individually
step 100 loss tensor([0.6872], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = than HIGH Protect Nine
step 200 loss tensor([0.5209], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = here strong lurk greeting
step 300 loss tensor([0.3947], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = Here Sil hello Hai
step 400 loss tensor([0.3651], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = HERE si hello hello
step 500 loss tensor([0.2650], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = There hello hello there
step 600 loss tensor([0.2291], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = HERE hello hello there
step 700 loss tensor([0.2019], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = WHAT hello hello there
step 800 loss tensor([0.1593], device='cuda:0', grad_fn=<RsubBackward1>)
	gen_text = WHAT hello hello there
s